In [ ]:
import sys

sys.path.insert(1, "../")

import glob
from dem_comparison.plots import plot_metrics
from pathlib import Path
from dem_comparison.utils import resample_dataset, read_metrics, filter_data
import multiprocessing as mp
import rasterio as rio
import numpy as np
from tqdm import tqdm
import psutil

In [ ]:
diffs = glob.glob(f"../Antarctica_Dem_Comparison/dem_diff/**.tif")
labels = ["ME", "STD", "MSE", "NMAD"]


def read_n_close(r):
    with rio.open(r, "r") as ds:
        data = ds.read(1)
        data = data[~np.isnan(data)]
    return data


count = 0
sum = 0
loop = tqdm(enumerate(diffs), total=len(diffs))
for i, d in loop:
    dt = read_n_close(d)
    count += len(dt)
    sum += dt.sum()
    loop.set_postfix(free_mem=psutil.virtual_memory().available / (1024**3))
mean = sum / count

std = 0
sum = 0
for i, d in loop:
    dt = read_n_close(d)
    sum += np.pow(dt - mean, 2).sum()
    loop.set_postfix(free_mem=psutil.virtual_memory().available / (1024**3))
std = np.sqrt(sum / count)

In [ ]:
is_error = True
polar = True
pkls = glob.glob(f"../Antarctica_Dem_Comparison/dem_diff_metrics/**.pkl")
custom_bins = [[-40, -20, -10, -7.5, -5, -2.5, 0, 2.5, 5, 7.5, 10, 20, 40], 15, 15, 15]
equal_bins = 5
no_bounds = None
no_percentile = None
normal_bounds = [(-50, 50), (0, 10), (0, 250), (0, 5)]
extreme_bounds = [(-10000, -10), (10, 50000), (1e4, 1e8), (2, 500)]
percentiles = (10, 98)
outliers = True
plot_name = "metric_plots/metrics_percentile_extremes_equal_bins.html"
plot_metrics(
    pkls,
    is_error,
    polar,
    plot_name,
    data_bounds=no_bounds,
    percentiles_bracket=percentiles,
    bins=equal_bins,
    plot_resolution=(700, 1600),
    percentile_outliers=not outliers,
)

In [ ]:
is_error = True
polar = True
pkls = glob.glob(f"../Antarctica_Dem_Comparison/dem_diff_metrics/**.pkl")
labels = ["ME" if is_error else "MEAN", "STD", "MSE", "NMAD"]
metrics, x0, y0 = read_metrics(pkls, numerical_axes=polar)
np.percentile(metrics[0], 10), np.percentile(metrics[0], 90)

In [ ]:
from dem_comparison.utils import simple_mosaic

tifs = glob.glob("../Antarctica_Dem_Comparison/overviews/*.tif")
simple_mosaic(tifs, "../Antarctica_Dem_Comparison/mosaic_downsampled.tif")

In [ ]:
# import rasterio as rio
# rio.open("../Antarctica_Dem_Comparison/mosaic_downsampled.tif").profile